In [82]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pandas as pd
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from BEKML import BEMKL, poly_kernel, gauss_kernel, scoring

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [ ]:
data = pd.read_csv('data/breast.csv', names=list(range(11)), index_col=0, na_values='?')
display(data.describe())
data = data.dropna()
display(data.head())
data.describe()

In [73]:
X = data.iloc[:, :-1].copy()
y = data.iloc[:, -1].copy()
N, D = X.shape
y.loc[y == 2] = -1
y.loc[y == 4] = 1
display(X.shape, y.shape, y.mean())

(683, 9)

(683,)

-0.3001464128843338

In [77]:
kernels = [lambda A, B: gauss_kernel(A, B, 2**i) for i in range(-3, 7)]
kernels += [lambda A, B: poly_kernel(A, B, 0, i) for i in range(1, 4)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i) for i in range(-3, 7) for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 0, i) for i in range(1, 4) for j in range(D)]
sparse_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1e-11, beta_omega=1e+9)
normal_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1, beta_omega=1)
len(kernels)

In [78]:
sparse_pipeline = make_pipeline(Normalizer(), sparse_model)
normal_pipeline = make_pipeline(Normalizer(), normal_model)

In [237]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=2)
cross_validate(sparse_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 130. Total: 130. Ratio: 1.0
1 - Non-0: 130. Total: 130. Ratio: 1.0
2 - Non-0: 130. Total: 130. Ratio: 1.0
3 - Non-0: 130. Total: 130. Ratio: 1.0
4 - Non-0: 130. Total: 130. Ratio: 1.0
5 - Non-0: 130. Total: 130. Ratio: 1.0


{'fit_time': array([ 10.1817,   9.4924,   8.5297,   8.5513,   8.5797,   9.0601]),
 'score_time': array([ 0.5394,  0.5378,  0.5184,  0.5298,  0.531 ,  0.5654]),
 'test_score': array([ 0.8772,  0.8772,  0.8722,  0.8728,  0.8816,  0.8899]),
 'train_score': array([ 0.9121,  0.9429,  0.9254,  0.9187,  0.9297,  0.9298])}

In [238]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=2)
cross_validate(normal_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 130. Total: 130. Ratio: 1.0
1 - Non-0: 130. Total: 130. Ratio: 1.0
2 - Non-0: 130. Total: 130. Ratio: 1.0
3 - Non-0: 130. Total: 130. Ratio: 1.0
4 - Non-0: 130. Total: 130. Ratio: 1.0
5 - Non-0: 130. Total: 130. Ratio: 1.0


{'fit_time': array([ 8.844 ,  8.8455,  8.8662,  8.7617,  8.788 ,  8.7561]),
 'score_time': array([ 0.5352,  0.5195,  0.5311,  0.5433,  0.5588,  0.5388]),
 'test_score': array([ 0.8816,  0.8772,  0.9295,  0.8991,  0.8991,  0.8899]),
 'train_score': array([ 0.9187,  0.9209,  0.9079,  0.9143,  0.9231,  0.8991])}

===================

In [269]:
kernels = [lambda A, B: gauss_kernel(A, B, 2**i) for i in range(-3, 7)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i) for i in range(1, 4)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i) for i in range(-3, 7) for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i) for i in range(1, 4) for j in range(D)]
sparse_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1e-11, beta_omega=1e+9)
normal_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1, beta_omega=1)
len(kernels)

130

In [270]:
sparse_pipeline = make_pipeline(Normalizer(), sparse_model)
normal_pipeline = make_pipeline(Normalizer(), normal_model)

In [271]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(sparse_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 130. Total: 130. Ratio: 1.0
1 - Non-0: 130. Total: 130. Ratio: 1.0
2 - Non-0: 130. Total: 130. Ratio: 1.0


{'fit_time': array([ 9.4544,  8.7845,  8.8687]),
 'score_time': array([ 0.5438,  0.5228,  0.529 ]),
 'test_score': array([ 0.8991,  0.8772,  0.8855]),
 'train_score': array([ 0.9187,  0.9407,  0.9211])}

In [272]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(normal_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 130. Total: 130. Ratio: 1.0
1 - Non-0: 130. Total: 130. Ratio: 1.0
2 - Non-0: 130. Total: 130. Ratio: 1.0


{'fit_time': array([ 8.7723,  8.475 ,  8.583 ]),
 'score_time': array([ 0.518 ,  0.5357,  0.5255]),
 'test_score': array([ 0.8991,  0.9079,  0.859 ]),
 'train_score': array([ 0.9165,  0.9209,  0.9254])}